In [1]:
import os
import sys

# Get the absolute path of the notebook's directory
notebook_dir = os.path.abspath(os.path.join(os.getcwd(), '..'))
sys.path.append(notebook_dir)

In [2]:
import torch
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from tqdm import tqdm
from src.app import PermutationSolver


In [3]:
n = 100
config = {
                'n_permutations_length': n,
                
                # Random walks params
                'random_walks_type': 'non-backtracking-beam',
                'n_random_walk_length': int(n * (n-1) / 2),
                'n_random_walks_to_generate': 1000,
                'n_random_walks_steps_back_to_ban': 8,
                
                # Neural Net params
                'model_type': 'MLP',
                'list_layers_sizes': [4096],
                'n_epochs': 100,
                'batch_size': 1024,
                'lr_supervised': 1e-3,
                
                # DQN training
                'n_epochs_dqn': 5000,
                'flag_dqn_round': False,
                'n_random_walks_to_generate_dqn': 1000,
                'verbose_loc': 50,
                'lr_rl': 1e-3,
                
                # Beam search
                'beam_search_torch': True,
                'beam_search_Fironov': False,
                'beam_width': 1,
                'n_steps_limit': 4 * n**2,
                'alpha_previous_cost_accumulation': 0,
                'beam_search_models_or_heuristics': 'model_torch',
                'ban_p0_p1_transposition_if_p0_lt_p1_ie_already_sorted': False,
                'n_beam_search_steps_back_to_ban': 32,
                
                # What to solve
                'solve_random_or_longest_state': 'solve_LRX_longest',
                'verbose': 100
            }

In [4]:
gpu_num = 1

n_perm_list = [12]
n_epoch_list = [100]
n_epoch_dqn_list = [100]
mlp_batch_size_list = [2**10]
dqn_batch_size_list = [2**16]
lr_supervised_list = [1e-3]
lr_rl_list = [1e-2]
n_rnd_walks_sup = [10_000]
n_rnd_walks_rl = [10_000]
anchor_list = [0]
beam_width_list = [2**0]
list_layers_sizes = [2**12]

summary_df = pd.DataFrame()

for n_permutations_length in n_perm_list:

    n = n_permutations_length
    config['n_permutations_length'] = n
    config['n_random_walk_length'] = int(n * (n-1) / 2)
    
    config['mode'] = 'single_hard_hinge'
    config['w_hinge'] = 1.0
    
    for anchor_mode in anchor_list:
        config['w_anchor'] = anchor_mode
        for n_rnd_walks_sup_mode in n_rnd_walks_sup:
            config['n_random_walks_to_generate'] = n_rnd_walks_sup_mode
            for n_rnd_walks_rl_mode in n_rnd_walks_rl:
                config['n_random_walks_to_generate_dqn'] = n_rnd_walks_rl_mode
                solver = PermutationSolver(config, gpu_id=gpu_num)
                solver.config['n_permutations_length'] = n_permutations_length
                for list_layers_size in list_layers_sizes:
                    solver.config['list_layers_sizes'] = [list_layers_size]
                    print(list_layers_size)
                    for lr in lr_supervised_list:
                        solver.config['lr_supervised'] = lr
                        for lr_rl in lr_rl_list:
                            solver.config['lr_rl'] = lr_rl
                            for mlp_batch_size in mlp_batch_size_list:
                                solver.config['mlp_batch_size'] = mlp_batch_size
                                for n_epoch in n_epoch_list:
                                    solver.config['n_epochs'] = n_epoch
                                    mlp_losses = solver.train_mlp(mode = config['w_anchor'])
                                    # save mlp_model
                                    torch.save(solver.mlp_model.state_dict(), f'models/mlp_model_{n_permutations_length}_{n_epoch}_{list_layers_size}_{lr}_{mlp_batch_size}.pth')
                                    for n_epoch_dqn in n_epoch_dqn_list:
                                        for dqn_batch_size in dqn_batch_size_list:
                                            solver.config['dqn_batch_size'] = dqn_batch_size
                                            solver.config['n_epochs_dqn'] = n_epoch_dqn
                                            dqn_losses = solver.train_dqn()
                                            # save dqn_model
                                            torch.save(solver.dqn_model.state_dict(), f'models/dqn_model_{n_permutations_length}_{n_epoch_dqn}_{list_layers_size}_{lr}_{dqn_batch_size}.pth')
                                            for beam_width in beam_width_list:
                                                solver.config['beam_width'] = beam_width
                                                i_step, flag_found_destination, path = solver.test_beam_search()
                
                                                # calculate min and max of difference between solver.y_anchor and y_valid
                                                # y_valid = solver.dqn_model(solver.X_anchor)
                                                device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
                                                solver.dqn_trainer.model.eval()               # set to eval mode
                                                y_parts = []
                                                n_samples = len(solver.X_anchor)
                                                with torch.no_grad():                 # <— no grad tracking
                                                    for start in range(0, n_samples, mlp_batch_size):
                                                        end = start + mlp_batch_size
                                                        X_batch = solver.X_anchor[start:end].to(device)
                                                        y_batch = solver.dqn_trainer.model(X_batch)
                                                        y_parts.append(y_batch.cpu()) # move back to CPU
                                                        # explicitly free GPU memory
                                                        del X_batch, y_batch
                                                        torch.cuda.empty_cache()
                                                
                                                y_valid = torch.cat(y_parts, dim=0)
                                                
                                                diff = solver.y_anchor.cpu().detach().numpy() - y_valid.cpu().detach().numpy().reshape(1, -1)[0]
                                                min_diff = diff.min()
                                                max_diff = diff.max()
                                                std_diff = diff.std()
                                                mean_diff = diff.mean()
                                                num_elements_less_than_minus_05 = (diff < -0.5).sum()
                                                num_elements_larger_than_05 = (diff > 0.5).sum()
                                                wrong_predictions = num_elements_less_than_minus_05 + num_elements_larger_than_05
                                                percentage_wrong_predictions = wrong_predictions / len(diff)
                
                                                # save summary_df
                                                summary_df_cur = pd.DataFrame({'n_permutations_length': n_permutations_length,
                                                                            'n_rnd_walks_sup_mode': n_rnd_walks_sup_mode,
                                                                            'n_rnd_walks_rl_mode': n_rnd_walks_rl_mode,
                                                                            'anchor':anchor_mode,
                                                                            'mlp_batch_size':mlp_batch_size,
                                                                            'dqn_batch_size':dqn_batch_size,
                                                                            'list_layers_sizes': [list_layers_size], 
                                                                            'lr_supervised': lr,
                                                                            'lr_rl': lr_rl, 
                                                                            'n_epoch': n_epoch, 
                                                                            'n_epoch_dqn': n_epoch_dqn, 
                                                                            'beam_width': beam_width, 
                                                                            'i_step': i_step, 
                                                                            'flag_found_destination': flag_found_destination, 
                                                                            'mlp_losses': mlp_losses[-1], 
                                                                            'dqn_losses': dqn_losses[-1],
                                                                            'min_diff': min_diff,
                                                                            'max_diff': max_diff,
                                                                            'mean_diff': mean_diff,
                                                                            'std_diff': std_diff,
                                                                            'num_elements_less_than_minus_05': num_elements_less_than_minus_05,
                                                                            'num_elements_larger_than_05': num_elements_larger_than_05,
                                                                            'percentage_wrong_predictions': percentage_wrong_predictions
                                                                            })
                                                summary_df = pd.concat([summary_df, summary_df_cur])
                                                summary_df.to_csv('models/summary_df.csv', index=False)

4096


RuntimeError: No CUDA GPUs are available

: 

In [5]:
summary_df['dqn_losses']

0    0.484506
Name: dqn_losses, dtype: float64

In [5]:
Добавь модель Кирилла
Сделай пуш
Поставь обучение на васт эй ай



SyntaxError: invalid syntax (1982195323.py, line 1)

In [ ]:
summary_dfd

,n_permutations_length,n_rnd_walks_sup_mode,n_rnd_walks_rl_mode,anchor,batch_size,list_layers_sizes,lr_supervised,lr_rl,n_epoch,n_epoch_dqn,...,flag_found_destination,mlp_losses,dqn_losses,min_diff,max_diff,mean_diff,std_diff,num_elements_less_than_minus_05,num_elements_larger_than_05,percentage_wrong_predictions
0,9,10000,10000,0,1024,4096,0.001,0.001,10,10,...,True,10.064562,0.047639,-3.053122,2.960304,0.117633,0.697093,69640,110067,0.500227
0,10,10000,10000,0,1024,4096,0.001,0.001,10,10,...,True,24.519262,0.117222,-4.568487,3.968890,-0.026837,0.956822,303835,300407,0.610345


In [11]:
solver.X_anchor.shape

torch.Size([36288, 9])

In [13]:
def factorial(n):
    """Calculate the factorial of a non-negative integer n."""
    if n < 0:
        raise ValueError("Factorial is not defined for negative numbers")
    if n == 0 or n == 1:
        return 1
    result = 1
    for i in range(2, n + 1):
        result *= i
    return result

factorial(9)

362880

In [6]:
18000/1000000

0.018

In [4]:
solver.generate_training_data_anchor()

362880
X.shape: torch.Size([362880, 9])
y.shape: torch.Size([362880])


(tensor([[0, 1, 2,  ..., 6, 7, 8],
         [1, 2, 3,  ..., 7, 8, 0],
         [8, 0, 1,  ..., 5, 6, 7],
         ...,
         [0, 8, 7,  ..., 3, 2, 1],
         [2, 1, 0,  ..., 5, 4, 3],
         [1, 0, 8,  ..., 4, 3, 2]], device='cuda:0'),
 tensor([ 0,  1,  1,  ..., 35, 35, 36], device='cuda:0'))

In [ ]:
import matplotlib.pyplot as plt

# Plot distribution of the torch vector y_valid
plt.figure(figsize=(10, 6))
plt.hist(diff, bins=30, alpha=0.7, color='blue')
plt.title('Distribution of y_valid')
plt.xlabel('Value')
plt.ylabel('Frequency')
plt.grid(True)
plt.show()